In [23]:
import pandas as pd
from parse import *
from scipy.stats import gmean
import numpy as np

def get_benchmark_list(file_name):
    benches = open(file_name).readlines()
    print("Number of benches :", len(benches))
    return [b.removesuffix('\n') for b in benches]

def parse_result_row_from_file(benchmark, directory):
    lines = open(directory + benchmark+".txt").readlines()
    ipc, instrucrions, cycles, accuracy, mpki = 0,0,0,0,0
    for line in lines:
        if "CPU 0 cumulative IPC" in line:
            # print("Fount line - " + (line))
            ipc, instrucrions, cycles = parse("CPU 0 cumulative IPC: {} instructions: {} cycles: {}\n", line)
            # print(ipc, instrucrions, cycles)
        if "CPU 0 Branch Prediction Accuracy" in line:
            # print("Fount line - " + (line) )
            accuracy, mpki, rob = parse("CPU 0 Branch Prediction Accuracy: {}% MPKI: {} Average ROB Occupancy at Mispredict: {}\n", line)
            # print(accuracy, mpki)
    return [benchmark.removesuffix(".champsimtrace.xz"), float(ipc), int(instrucrions), int(cycles), float(accuracy), float(mpki)]


def main():
    predictors = ['bimodal', 'perceptron', 'gshare']
    variations = 3
    benches = get_benchmark_list("sims")
    
    
    
    for predictor in predictors:
        for variant in range(1,variations+1):
            results = pd.DataFrame( {
                "Benchmark": [],
                "IPC":[],
                "instructions": [],
                "cycles":[],
                "accuracy": [],
                "MPKI":[] })
            index = 0
            directory = "../results/out/" + predictor + "_" + str(variant) + "/"
            for benchmark in benches:
                
                results.loc[index] = parse_result_row_from_file(benchmark, directory)
                index = index+1
     
            # print(results.iloc[:, 1:].to_numpy()) 
            mean = gmean(results.iloc[:, 1:], axis=0)
            results.loc[index] = ["mean"] + mean.tolist()
            print( results)
            results.to_csv(directory + predictor + "_" + str(variant) + ".csv", index=False)
            
    
main()

Number of benches : 20
<class 'numpy.ndarray'>
                Benchmark       IPC  instructions        cycles   accuracy  \
0    600.perlbench_s-210B  1.565950  1.000000e+09  6.385885e+08  96.663900   
1          602.gcc_s-734B  0.556032  1.000000e+09  1.798459e+09  94.656200   
2      603.bwaves_s-3699B  1.632610  1.000000e+09  6.125144e+08  80.516900   
3          605.mcf_s-665B  0.429728  1.000000e+09  2.327056e+09  88.277800   
4   607.cactuBSSN_s-2421B  1.025450  1.000000e+09  9.751857e+08  99.698400   
5         619.lbm_s-4268B  0.296085  1.000000e+09  3.377403e+09  97.806700   
6      620.omnetpp_s-874B  0.532344  1.000000e+09  1.878486e+09  95.462800   
7          621.wrf_s-575B  0.905487  1.000000e+09  1.104378e+09  95.788700   
8    623.xalancbmk_s-700B  0.894049  1.000000e+09  1.118507e+09  97.141400   
9          625.x264_s-18B  2.360600  1.000000e+09  4.236220e+08  85.776800   
10        627.cam4_s-573B  1.014190  1.000000e+09  9.860052e+08  90.156800   
11         628.po